# Dimension reduction answers

Notebook containing the example answers to Chapter 5 - Clustering.

Note: these answers are examples and your way of tackling the problem may also be correct.

## Preparation

Please run this first.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram

## Exercise 1

Using the data imported below, visualise X. 

Without looking at the true label values perform K-means clustering on the data with $K=2$. 

Visualise the results of the clustering labels assigned as shown in previous examples. Change the value of $K$ to a more appropriate value and compare the results.


In [ ]:
# Import and separate data
unknown_clusters_data = np.loadtxt("../data/exercise_one_clusters.csv", delimiter=",")
X, _true_labels= unknown_clusters_data[:,0:-1], unknown_clusters_data[:,-1]

In [ ]:
# Write your code here
# Visualise X

X.shape

In [ ]:
# Looking at the data without class labels to understand it's structure
plt.scatter(x=X[:,0], y=X[:,1]);

X has two dimensions of data, there are some clear groupings of data points, and other areas where potential groupings are near each other.

In [ ]:
# Perform the clustering, produce group labels

# Create and fit the clustering
kmeans_2_clusters = KMeans(n_clusters=2).fit(X)

# Access the found labels
found_clusters = kmeans_2_clusters.labels_

In [ ]:
# Visualise your clustering results

plt.title("K=2 Found Labels")
plt.scatter(x=X[:,0], y=X[:,1], c=found_clusters);

We can see that this is quite a poor clustering. We have split the data into two labelled classes, which doesn't appear appropriate for the data. 

In [ ]:
# Find a better value of K
# Repeating the code above and changing the value of `n_clusters`, visualising the results.

K = 5

kmeans_2_clusters = KMeans(n_clusters=K).fit(X)

found_clusters = kmeans_2_clusters.labels_

plt.scatter(x=X[:,0], y=X[:,1], c=found_clusters);

Without using a way to evaluate the performance of a clustering, we can only really visualise and compare groupings using our intuition. Using just `X` and KMeans clustering there appears to be five different clusters in the data set.

## Exercise 2
Using the data imported below and the ARI work out what the best value for $K$ is. Plot the results of the best $K$. 

Using the true labels work out how many actual groups in the data there are, is this the same as what you found? Plot the true labels and compare results.

**HINT** The np.unique() function gives you the unique elements in an array. Remember, the exact numerical label of each cluster isn't important, but whether points belong to the same group.

In [ ]:
# Import and separate data
unknown_clusters_data = np.loadtxt("../data/exercise_two_clusters.csv", delimiter=",")
X, true_labels= unknown_clusters_data[:,0:-1], unknown_clusters_data[:,-1]

In [ ]:
# Write your code below
# Find the best value for K using ARI

# Create list to store value of K and associate ARI score
ARI_scores = []

# Generate values of K to be looped through
K_values = range(1, 20)

# Loop though values of K, creating new model and evaluating for each
for K in K_values:
    KMeans_model = KMeans(n_clusters=K).fit(X)
    found_clusters = KMeans_model.labels_
    # Calculate evaluation score
    ARI_score = adjusted_rand_score(true_labels, found_clusters)
    # Store K and ARI
    ARI_scores.append((K, ARI_score))

# Find the highest value ARI score and associated K
best_K, best_score = max(ARI_scores, key=lambda x:x[1])

In [ ]:
print("Best K value:", best_K)
print("ARI of best K:", round(best_score, 4))

In [ ]:
# Plot the found labels for your value of K

best_KMeans_model = KMeans(n_clusters=best_K_values[0]).fit(X)
best_found_clusters = best_KMeans_model.labels_

plt.scatter(x=X[:,0], y=X[:,1], c=best_found_clusters);

This clustering looks very reasonable, with each group compact and separate.

It would also be useful to look at the different ARI scores of each value of $K$.

In [ ]:
# Calculate how many true labels there are
unique_true_labels = np.unique(true_labels)

# Given the unique labels, how many labels is the length of the array
print("Number of true classes:", len(unique_true_labels))

This is close to the number of classes we found using ARI, but not exactly the right number. 

In [ ]:
fig, axes = plt.subplots(figsize=(12,5), ncols=2, nrows=1, sharey=True)

axes[0].set_title("True Labels")
axes[0].scatter(x=X[:,0], y=X[:,1], c=true_labels, cmap="magma")

axes[1].set_title("Best Found Labels")
axes[1].scatter(x=X[:,0], y=X[:,1], c=best_found_clusters, cmap="viridis");

We can see two main things from this comparision.

Firstly, it shows why we have such a high ARI value, our clustering is getting most of the groupings correct. In each cluster, there are some errors or miss-grouped selections around edges, this is natural.

Secondly we can see why we have found 7 clusters instead of 8. In the True Labels figure there are two clusters that are occupying a significant amount of common space. Without knowing the true labels it is very difficult for an algorithm to differentiate between these two clusters. Some methods are better at this than others.

The effect this may have on our analysis is underestimating the number of groups in our data set. More importantly as a result we may lose two unique group in our analysis, and instead have one larger group.

For example if we were clustering people's exercise habits, we could end up finding the group "people who play racket sports" rather than "people who play tennis" and "people who play table tennis".

Without the true label values, it would be challenging to work this out. This is where domain knowledge and a thorough understanding of the data come into play. 

## Exercise 3

Using the data imported below, perform KMeans clustering with $K=5$, calculate the silouette score using both manhattan and euclidean distances.

Plot both found label sets, are there significant differences?

In [ ]:
# Load the data
unknown_clusters_data = np.loadtxt("../data/exercise_three_clusters.csv", delimiter=",")
X, _true_labels = unknown_clusters_data[:,0:-1], unknown_clusters_data[:,-1]

In [ ]:
# Write your code below
# Perform Kmeans clustering on the data.

KMeans_model_5 = KMeans(n_clusters=5).fit(X)

In [ ]:
# Generate labels and silhouette score using both metrics
found_clusters = KMeans_model_5.labels_

# Score using each metric
sil_score_euc = silhouette_score(X=X, labels=found_clusters, metric="euclidean")
sil_score_man = silhouette_score(X=X, labels=found_clusters, metric="manhattan")

print("Euclidean Silhouette Score:", round(sil_score_euc, 4))
print("Manhattan Silhouette Score:", round(sil_score_man, 4))

Whilst we do not select a distance metric to perform the KMeans clustering itself, we can do so when evaluating our clusterings intrinsically. In this case, there is not a significant difference between the scores of Manhattan and Euclidean metrics, but as have more complex data this may not always be true. As with all modelling packages it is important to check what the default parameters are before relying on the results!

## Exercise 4

Using the data loaded below, using *eps=0.15* calculate the silhouette score of the DBSCAN clustering.


In [ ]:
# Load the data
swirl_data = np.loadtxt("../data/exercise_four_clusters.csv", delimiter=",")
X, _true_labels= swirl_data[:,0:-1], swirl_data[:,-1]

In [ ]:
# Write your code below

# Define the value for eps
reach_dist = 0.15

# Create and fit the model
dbscan_clusterer = DBSCAN(eps=reach_dist, metric="euclidean").fit(X)
found_clusters = dbscan_clusterer.labels_

# Score the found labels
sil_score_euc = silhouette_score(X=X, labels=found_clusters, metric="euclidean")

print("eps={} silhouette score: {:.4f}".format(reach_dist, sil_score_euc))

In [ ]:
# Visualise the found clusters
plt.scatter(x=X[:,0], y=X[:,1], c=found_clusters);

This appears to be a well grouped clustering, however, there is a low silhouette score. This is a result of the choice in evaluation metric we have used. A high silhouette score favours compact, well distanced clusters. Because our data classes are *not well distanced or compact*, even the perfect clustering assignment would have a poor silhouette score.

This is one reason why selecting an appropriate measurement is important. In addition, we should measure model performances *relative to each other and with the data in mind*.  

## Exercise 5

Using the data loaded below, visualise the two true classes in the data set, why might these clusters be challenging to model?

Select a distance metric to use, then find an appropriate value of *eps* to use with DBSCAN using an appropriate evaluation method. Why did you select this distance metric?

**HINT:** Check how many dimensions are in the data.

In [ ]:
# Load the data
further_swirl_data = np.loadtxt("../data/exercise_five_clusters.csv", delimiter=",")
X, true_labels= further_swirl_data[:,0:-1], further_swirl_data[:,-1]

In [ ]:
X.shape

In [ ]:
# Write your code below
# Visualise the data

fig, axes = plt.subplots(figsize=(12,5), ncols=2)

axes[0].scatter(x=X[:,0], y=X[:,1], c=true_labels, cmap="coolwarm")
axes[1].scatter(x=X[:,0], y=X[:,2], c=true_labels, cmap="coolwarm");

Looking at the first figure, the two classes are highly intertwinned. In addition the have similar densities and are not well separated, with wide spread. 

The second image however shows that the data is quite well separated in the X[:,2] dimensions (third column of data), this could be useful to group the classes.

It is important that we use all of the data at our disposal to group the data. 

Having tweaked the value of `eps` for both "euclidean" and "manhattan" distances, they can both find an optimal value of `eps` maximising performance. An important point to note is that the `eps` found for each distance metric will be different, as the reach distance is a different quantity.

Which to pick here is a bit of a trick question, there are use cases for manhattan distance, but by default we tend to stick to euclidean.

In [ ]:
# First explore the sensitivity to `eps` the data has and visualise the results. 
# We can optimise on some evaluation score shortly

# Feel free to change `reach_dist` to explore impact of value
reach_dist = 0.25

dbscan_clusterer = DBSCAN(eps=reach_dist, metric="euclidean").fit(X)
found_clusters = dbscan_clusterer.labels_

In [ ]:
fig, axes = plt.subplots(figsize=(12,5), ncols=2)

axes[0].scatter(x=X[:,0], y=X[:,1], c=found_clusters, cmap="plasma")
axes[1].scatter(x=X[:,0], y=X[:,2], c=found_clusters, cmap="plasma");

# Count how many unlabelled data pounts exist in the data set
# Unlabelled data == -1, we can create a mask then count the True's (1's)
print("Number unlabelled", np.count_nonzero(found_clusters == -1))

I encourage selecting different values of `eps` manually to find an appropriate performing reaching distance.

For this task, the Adjusted Rand Index would be helpful, as we have the true labels. However, as we typically cluster when we do not have a true label value, we are going to use an intrinsic method instead.

As shown in the previous exercises, the silhouette score performs poorly when the data groups are not compact (convex). This, unfortunately is an inherent problem with density based clustering. We should still be able to find the best value. For more evaluation options in `sklearn` [look at the documentation](https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation).

In [ ]:
# Define the range of eps values
eps_values = np.linspace(start=0.01, stop=1, num=200)

# A list to store eps values and associate scores
sil_scores = []

# Specify the metric
distance_metric = "manhattan"

# Loop over all eps values defined
for reach_dist in eps_values:
    
    DBSCAN_model = DBSCAN(eps=reach_dist, metric=distance_metric).fit(X)
    found_clusters = DBSCAN_model.labels_
    # silhouette score only defined when more than one cluster found
    if len(np.unique(found_clusters)) > 1:
        sil_score = silhouette_score(X, found_clusters)
        sil_scores.append((reach_dist, sil_score))

In [ ]:
# Find eps with highest associated silhouette score
highest_eps, highest_value = max(sil_scores, key=lambda x:x[1])

print("Best value of eps:", round(highest_eps, 4))
print("Best eps Silhouette Score:", round(highest_value, 4))

In [ ]:
best_DBSCAN = DBSCAN(eps=highest_eps, metric=distance_metric).fit(X)
best_found_clusters = best_DBSCAN.labels_

fig, axes = plt.subplots(figsize=(12,5), ncols=2)

axes[0].scatter(x=X[:,0], y=X[:,1], c=best_found_clusters, cmap="copper")
axes[1].scatter(x=X[:,0], y=X[:,2], c=best_found_clusters, cmap="copper");

print("Number unlabelled", np.count_nonzero(best_found_clusters == -1))

We can see that this optimal value of `eps` found separates the two clusters very well for what we wanted, despite the low silhouette values. 

## Exercise 6

Using the "three_clusters_data" imported below, cluster the data with $n\_clusters=3$ using both single and complete-linkage with the AgglomerativeClustering model. For both results, calculate the silhouette score. Which is higher? Why do you think this is?

In [ ]:
from sklearn.cluster import AgglomerativeClustering
three_clusters_data = np.loadtxt("../data/clusters.csv", delimiter=",")
X, true_labels = three_clusters_data[:,0:-1], three_clusters_data[:,-1]

In [ ]:
# Write your code below

# Lets first view our data
plt.scatter(x=X[:,0], y=X[:,1]);

In [ ]:
# Create model for each linkage method
single_linkage_model = AgglomerativeClustering(n_clusters=3, linkage="single").fit(X)
complete_linkage_model = AgglomerativeClustering(n_clusters=3, linkage="complete").fit(X)

# Access labels assigned for each method
single_clusters = single_linkage_model.labels_
complete_clusters = complete_linkage_model.labels_

# Evaluate performance of each model and ground truth
single_silhouette = silhouette_score(X, single_clusters)
complete_silhouette = silhouette_score(X, complete_clusters)
true_silhouette = silhouette_score(X, true_labels)


print("Single score:", round(single_silhouette, 4))
print("Complete score:", round(complete_silhouette, 4))
print("'True' score:", round(true_silhouette, 4))

We can see from the difference in silhouette score that the complete linkage method performs much better than the single linkage. In fact, compared to the best possible silhouette score (using the true labels) the complete linkage performs *extremely* well in this toy example. 

In [ ]:
fig, axes = plt.subplots(figsize=(12,5), ncols=2)

axes[0].set_title("Single Linkage")
axes[0].scatter(x=X[:,0], y=X[:,1], c=single_clusters, cmap="viridis")

axes[1].set_title("Complete Linkage")
axes[1].scatter(x=X[:,0], y=X[:,1], c=complete_clusters, cmap="viridis");

In the above figure it is clear that the single-linkage has grouped two of the large clusters together, and kept a single point as a cluster. This has happened because one single point is further away from any other cluster than any of the points within any other cluster.

In this sense, the single-linkage can be very susceptible to statistical noise, and cannot reliable give us the cluster groupings we want. 

Where our data is compact, the complete linkage can perform very well, such as in the example above.

## Exercise 7

Why might agglomerative clustering be slow / resource demanding to impliment?

---

In a naive implementation of the hierarchical clustering described, the time taken to complete the modelling will increase significantly as more data is added.

When the algorithm starts, the distance between every single data point needs to be calculated to every other data point. If we have $N$ data points that is $N^2$ distance calculations just for the first step. 

For a few hundred/thousand data points each with a couple dimensions this is easy for our computers. However, as we increase the dimensionality of our data (columns), and the number of points (rows), the number of calculations increases rapidly. 

Thankfully, most clustering algorithms do not need to calculate every single step and can take some shortcuts to bypass some computing effort. The more shortcuts we make however, the higher the potential for less accurate results. 

As a result of the number of calculations, some algorithms can become slow or impossible to run on large data. 

For hierarchical clustering we need to store information about the clustering at each step as each merge occurs, this can take up a large amount of computer memory. 

## Exercise 8

Using the "three_clusters_data" imported below, cluster the data with $n\_clusters=3$ use single linkage and the *scipy* package. 

Calculate the silhouette score of this clustering.

In [ ]:
three_clusters_data = np.loadtxt("../data/clusters.csv", delimiter=",")
X, labels_true = three_clusters_data[:,0:-1], three_clusters_data[:,-1]

In [ ]:
# Write your code below
# Calculate the linkage matrix
single_linkage_matrix = linkage(X, 'single')

# Flatten the linakge matrix to the required number of clusters
found_single_labels = fcluster(single_linkage_matrix,
                               criterion="maxclust", t=3)

In [ ]:
single_score = silhouette_score(X, found_single_labels)
print("Single Linkage silhouette score", round(single_score, 4))

## Exercise 9

Plot a dendrogram using the data loaded below, previously used in exercise one, and Ward linkage. 

Looking at the dendrogram, what appears to be a sensible number of clusters to take?

Calculate the silhouette value of this number of clusterings.

In [ ]:
# Import and separate data
unknown_clusters_data = np.loadtxt("../data/exercise_one_clusters.csv", delimiter=",")
X, _true_labels= unknown_clusters_data[:,0:-1], unknown_clusters_data[:,-1]

In [ ]:
# Write your code here

# Calculate the linkage matrix
ward_linkage_matrix = linkage(X, 'ward')


# Create the dendrogram object, show the counts and specify the method of truncation
average_dendrogram = dendrogram(ward_linkage_matrix, 
                                show_leaf_counts=True, 
                                p=10, 
                                truncate_mode="lastp")
plt.title("Dendrogram of Ward Linkage")
plt.ylabel("Merge Distance");

Looking at the dendrogram we can see that there is a significant amount of distance between the merging of the final two clusters. There too are significant gaps between merges down until there are 5 clusters. After 5 clusters the distance to join successive clusters decreases. This implies that 5 clusters may be a good choice.

In [ ]:
# Flatten the linkage matrix to the required number of clusters
found_ward_labels = fcluster(ward_linkage_matrix,
                               criterion="maxclust", t=5)

# Calculate the performance
ward_score = silhouette_score(X, found_ward_labels)

print("Ward silhouette score:", round(ward_score, 4))

This number of clusters appears to perform well, but why not check above and below to make sure well didn't misinterpret the figure.

In [ ]:
# Loop through a range of number of clusters
for number_of_clusters in range(2, 9):
    # Flatten the linkage matrix to the specified number of clusters
    found_ward_labels = fcluster(ward_linkage_matrix,
                                 criterion="maxclust", 
                                 t=number_of_clusters)

    ward_score = silhouette_score(X, found_ward_labels)
    print("{} clusters, score: {:.4}".format(number_of_clusters, ward_score))

We can see that with regards to our evaluation method, 5 clusters is the best. 